In [ ]:
import numpy as np
import pandas as pd
import librosa, librosa.display
import matplotlib.pyplot as plt
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import collections
import pprint
from os import path
import json
import os
import codecs
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import MinMaxScaler




In [ ]:
tf.__version__

In [ ]:
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

In [ ]:
#read csv file
with open('./annotations.csv', 'r') as f:
  df = pd.read_csv(f,delimiter='\t')

In [ ]:
#sort
df = df.sort_values(by=['mp3_path', 'clip_id'])
df

In [ ]:
temp_df = df.drop(columns=['clip_id', 'mp3_path'])

In [ ]:
#Top 50 tags
lst = []
for i in temp_df.columns:
  a = np.sum(np.array(df[i]))
  lst.append(a)

top_50_indexes = np.argsort(-np.array(lst))[:50]
new_df = temp_df.iloc[:,top_50_indexes].copy()


In [ ]:
test = {'labels': []}

for i, j in new_df.iterrows():
  test['labels'].append(list(j[0:50]))

In [ ]:
range_a = 0
range_b = 0
df = df['mp3_path'].values

for number in range(len(df)):
  if df[number][0] == 'c':
    range_a = number
    break

for number in range(range_a+1, len(df)):
  if df[number][0] == 'd':
    range_b = number
    break






In [ ]:
print(range_a)
print(range_b)

In [ ]:
dataset_path = './music'
#dataset_path = '/content/drive/MyDrive/MTAT/test_sample'
json_path = "/content/srpa_d.json"
track_duration = 29 # measured in seconds  why is 29 instead of 30?

In [ ]:
for i in sorted(os.listdir(dataset_path)):
  print(i)

In [ ]:
count = 0
for i in sorted(os.listdir(dataset_path)):
  count += 1

print(count)

In [ ]:
scaler = MinMaxScaler()
def upload_data(dataset_path, hop_length=512, num_segments=5, n_fft=2048):

  data = {
      'train': {
      #'Labels': [],
      'Mel_Spectrogram': []
      },
      'validation': {
      #'Labels': [],
      'Mel_Spectrogram': []
      },
      'test': {
      #'Labels': [],
      'Mel_Spectrogram': []   
      }
  }

  mul = 1
  
  for folder in sorted(os.listdir(dataset_path)[1:]):
      print(folder)
      counter = 0
      for file in sorted(os.listdir(dataset_path + '/' + folder )):
          print(file)
          if file.startswith('.'):
            continue
            
          if counter == 200*mul:
              break
          counter += 1 

          signal, sr = librosa.load(dataset_path + '/' + folder + '/' + file)

          samples_per_segment = (sr*track_duration)/num_segments
          num_mel_specs_file = samples_per_segment/hop_length
          print(counter)
          for i in range(num_segments):
              start = int(samples_per_segment * i)
              finish = int(start + samples_per_segment)
              spectrogram = librosa.feature.melspectrogram(signal[start:finish], sr, n_fft=n_fft, hop_length=hop_length, n_mels=20)

              log_spectrogram = scaler.fit_transform(librosa.power_to_db(spectrogram))

              if folder < 'c':
                data['train']['Mel_Spectrogram'].append(log_spectrogram.tolist())
              elif folder == 'c':
                data['validation']['Mel_Spectrogram'].append(log_spectrogram.tolist())
          #else:
            #data['test']['Mel_Spectrogram'].append(log_spectrogram.tolist())
      mul = .1
     
    
  #add it to json file 
  with open('./data.json', 'w') as f:
    json.dump(data, f)
    
    
    #spectrogram = librosa.feature.melspectrogram(y=y, sr=sr) #should the spectrogram be transposed? 
    #Store as JSON file(s)?
    # store the name/id
    #with open(json_path, "w") as fp:
        #json.dump(data, fp, indent=4)
      

In [ ]:
json_file = upload_data(dataset_path)

In [ ]:
def load_data(json_path, subset):
  with open(json_path,"r") as f:
    file = json.load(f)

  x = file[subset]['Mel_Spectrogram']
  #y = file[subset]['Labels']
  print(x)
  #print(y)

  return x#, y


In [ ]:
inputs = load_data("data.json", 'train')
inputs_valid = load_data("data.json", 'validation')

In [ ]:
print(np.expand_dims(np.array(inputs), axis=-1).shape)

In [ ]:
df_train = pd.DataFrame(np.repeat(new_df.head(200).values, 5, axis=0))
df_valid = pd.DataFrame(np.repeat(new_df.head(20).values, 5, axis=0))

In [ ]:
x_train=np.expand_dims(np.array(inputs), axis=-1)
y_train=np.array(df_train)


x_valid = np.expand_dims(np.array(inputs_valid), axis=-1)
y_valid = np.array(df_valid)

In [ ]:
print(x_train.shape)
print(x_valid.shape)

In [ ]:
#implementation without keras

In [ ]:
x = tf.placeholder("float32", [None, 20, 250,1])
y = tf.placeholder("float32", [None, 50])  

In [ ]:
def conv2d(x, w, b):
  x = tf.nn.conv2d(x,w, padding='SAME')
  x = tf.nn.bias_add(x,b)
  #x = tf.layers.batch_normalization(x, axis=0)
  x, _ = tf.linalg.normalize(x)
  return tf.nn.relu(x)

In [ ]:
def max_p_2d(x, k1, k2):
  return tf.nn.max_pool2d(x, ksize=[1,2,2,1], strides=[1, k1, k2, 1], padding='SAME')

In [ ]:
weights = {
    'wc1': tf.get_variable('w1', shape=(3,3,1,128), initializer=tf.random_normal_initializer, dtype=tf.float32),
    'wc2': tf.get_variable('w2', shape=(3,3,128,384), initializer=tf.random_normal_initializer, dtype=tf.float32),
    'wc3': tf.get_variable('w3', shape=(3,3,384,768), initializer=tf.random_normal_initializer, dtype=tf.float32),
    'wd1': tf.get_variable('wd1', shape=(3*32*768, 2048), initializer=tf.random_normal_initializer, dtype=tf.float32),
    #'wc4': tf.get_variable('w4', shape=(3,3,768,2048), initializer=tf.random_normal, dtype=tf.float64),
    'out': tf.get_variable('out', shape=(2048, 50), initializer=tf.random_normal_initializer, dtype=tf.float32)
}

biases = {
    'bc1': tf.get_variable('b1', shape=(128), initializer=tf.random_normal_initializer, dtype=tf.float32),
    'bc2': tf.get_variable('b2', shape=(384), initializer=tf.random_normal_initializer, dtype=tf.float32),
    'bc3': tf.get_variable('b3', shape=(768), initializer=tf.random_normal_initializer, dtype=tf.float32),
    'bc4': tf.get_variable('b4', shape=(2048), initializer=tf.random_normal_initializer, dtype=tf.float32),
    'out': tf.get_variable('b5', shape=(50), initializer=tf.random_normal_initializer, dtype=tf.float32)
}

In [ ]:
def CNN(x, weights, biases):

  conv1 = conv2d(x, weights['wc1'], biases['bc1'])

  conv1 = max_p_2d(conv1, 2, 2)
  #conv1 = tf.nn.dropout(conv1, rate = 0.5)
 
  conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
  conv2 = max_p_2d(conv2, 2, 2)
  #conv2 =  tf.nn.dropout(conv2, rate = 0.5)

  conv3 = conv2d(conv2, weights['wc3'], biases['bc3'])
  conv3 = max_p_2d(conv3, 2, 2)
  #conv3 =  tf.nn.dropout(conv3, rate = 0.5)
  print(conv3.shape)

  fc1 = tf.reshape(conv3, [-1, weights['wd1'].get_shape().as_list()[0]])
  fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bc4'])
  fc1 = tf.nn.relu(fc1)
  fc1 = tf.nn.dropout(fc1, rate=0.5)
  print(fc1.shape)

  #conv4 = conv2d(conv3, weights['wc4'], biases['bc4'])
  #conv4 = max_p_2d(conv4, 2, 5)
  #conv4 = tf.nn.dropout(conv4, rate = 0.5)

  #print(conv4.shape)
  #out = tf.add(tf.matmul(tf.squeeze(conv4,[1]), weights['out']), biases['out']
  #out = tf.compat.v1.layers.flatten(out)
  out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
  #print(out)
  return out

In [ ]:
learning_rate = 0.0001
batch_size = 4  
epochs = 100

In [ ]:
"""num_labels = len(y_train.flatten())
num_pos_labels = np.sum(y_train.flatten())
num_neg_labels = num_labels-num_pos_labels
pos_weight = num_pos_labels/num_neg_labels
print(pos_weight)"""

In [ ]:
pred = CNN(x, weights, biases)

#loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=y, logits=pred))
#loss = tf.losses.log_loss(labels=y, predictions=pred)
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=pred))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)
accuracy = tf.metrics.auc(labels=y, predictions=tf.round(tf.cast(tf.nn.sigmoid(pred), dtype=tf.int16)))

In [ ]:
## Initializing the variables
#init = tf.global_variables_initializer()
tf.disable_eager_execution()
init = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())


with tf.Session() as sess:
    sess.run(init)
    for i in range(epochs):
      train_loss = []
      train_accuracy = []
      valid_accuracy = []
      #for num in range(x_train.shape[0]):
      print(x_train.shape[0])
      for num in range(0,(x_train.shape[0]//batch_size)*batch_size,batch_size):
        batch_x = x_train[num:num+batch_size,:,:]
        batch_y = y_train[num:num+batch_size,:] 
        #test = sess.run(pred, feed_dict={x:batch_x})
        #print(test)
        #print(sess.run(tf.round(tf.cast(tf.nn.sigmoid(test).eval(), dtype=tf.int16))))

        _, loss_ = sess.run([optimizer, loss], feed_dict={x: batch_x, y:batch_y})
        acc = sess.run(accuracy, feed_dict={x:batch_x, y:batch_y})
        
    
        train_accuracy.append(acc)
        train_loss.append(loss_)
        
      print('epoch ',i)
      print('accuracy: {}'.format(np.mean(train_accuracy)))
      print('loss: {}'.format(np.mean(train_loss)))
      
      for num in range(0,(x_valid.shape[0]//batch_size)*batch_size,batch_size):
        batch_x = x_valid[num:num+batch_size,:,:]
        batch_y = y_valid[num:num+batch_size,:] 
        
        acc = sess.run(accuracy, feed_dict={x:batch_x, y:batch_y})
        valid_accuracy.append(acc)
        
      print('accuracy: {}'.format(np.mean(valid_accuracy)))